In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [17]:
from ml.preprocess import data
from utils import helper, config
from ml.models.base.slug_ann import SlugANN
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from dask_ml.preprocessing import StandardScaler

In [21]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)
X_scalar = StandardScaler().fit_transform(X)

X_base, X_test, y_base, y_test = train_test_split(X_scalar, y, test_size=0.33)
X_train, X_val, y_train, y_val = train_test_split(X_base, y_base)




In [22]:
slug_ann = SlugANN()
slug_ann.X_train = X_train
slug_ann.X_test = X_test
slug_ann.X_val = X_val
slug_ann.y_train = y_train
slug_ann.y_test = y_test
slug_ann.y_val = y_val

slug_ann.n_trials = 5    #### increase this number to imporve the goodness of fit
slug_ann.epochs = 100

In [7]:
best_ann = slug_ann.fetch_model()

2022-10-12 11:25:51,567 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-5964_5kj', purging
2022-10-12 11:25:51,567 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-uyhstbda', purging
2022-10-12 11:25:51,567 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-nzr3lvxn', purging
2022-10-12 11:25:51,567 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-e5r_lc34', purging


Dask dashboard is available at http://127.0.0.1:8787/status
Starting train for trials:5 with epochs:100
Number of trials: 5
Best trial:
Model saved at:/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/models/saved/base/ann/slug/brisk_ann_0.pickle


In [8]:

# use any of the score functions of sklearn.metircs, pass a list
slug_ann.get_model_score(score_func=[r2_score, mean_squared_error])

[-7.8184266458136396, 2998304.889871754]

In [48]:
file_name = config.main_dir + '/ml/models/saved/base/brisk_nn_pytorch/brisk_nn_pytorch.pickle'
file_name

'/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/models/saved/base/brisk_nn_pytorch/brisk_nn_pytorch.pickle'

In [49]:
best_ann = slug_ann.load_model(file_name)

In [50]:
### Test score
X_test_tensor = helper.df_to_tensor(X_test)
pred = best_ann(X_test_tensor)
pred = helper.torch_tensor_to_numpy(pred)
r2_score(pred, y_test), mean_squared_error(pred, y_test)



(0.8677055915087115, 146324.98499475874)

In [51]:
### Train score
X_train_tensor = helper.df_to_tensor(X_train)
pred = best_ann(X_train_tensor)
pred = helper.torch_tensor_to_numpy(pred)
r2_score(pred, y_train), mean_squared_error(pred, y_train)


(0.786116729866593, 262988.35601034906)